In [1]:
import pandas as pd
import bcolz
import pickle
import numpy as np

## 데이터 로딩하고 pandasd.series를 list로 변경하는 곳

In [2]:
data = pd.read_csv("./train_final.csv")
X_train_txt = data["Sentence"]
y_train_num = data["Category"]

X_train_txt = pd.Series.tolist(X_train_txt)
y_train_num = pd.Series.tolist(y_train_num)

## 데이터 토큰화

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\owner\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from nltk.tokenize import word_tokenize
print(word_tokenize(X_train_txt[0]))

['-LRB-', 'The', 'film', '-RRB-', 'tackles', 'the', 'topic', 'of', 'relationships', 'in', 'such', 'a', 'straightforward', ',', 'emotionally', 'honest', 'manner', 'that', 'by', 'the', 'end', ',', 'it', "'s", 'impossible', 'to', 'ascertain', 'whether', 'the', 'film', 'is', ',', 'at', 'its', 'core', ',', 'deeply', 'pessimistic', 'or', 'quietly', 'hopeful', '.']


In [5]:
X_train_txt_tocken = []
for i,data in enumerate(X_train_txt):
    X_train_txt_tocken.append(word_tokenize(data))

In [6]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train_txt_tocken)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train_txt_tocken))/len(X_train_txt_tocken)))

리뷰의 최대 길이 : 53
리뷰의 평균 길이 : 19.13842688842689


* 평균 길이보다 2배정도 긴 40으로 padding 수행

## pretrained된 glove사전을 가져와 사용하기

In [7]:
words = []
idx = 0
word2idx = {}
glove_path = "C:/Users/owner/Documents/GIT/Deeplearning_han/kyss"
vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/6B.100.dat', mode='w')

with open(f'{glove_path}/glove.6B.100d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 100)), rootdir=f'{glove_path}/6B.100.dat', mode='w')
vectors.flush()

In [8]:
pickle.dump(words, open(f'{glove_path}/6B.100_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'{glove_path}/6B.100_idx.pkl', 'wb'))

In [9]:
vectors = bcolz.open(f'{glove_path}/6B.100.dat')[:]
words = pickle.load(open(f'{glove_path}/6B.100_words.pkl', 'rb'))
word2idx = pickle.load(open(f'{glove_path}/6B.100_idx.pkl', 'rb'))

glove = {w: vectors[word2idx[w]] for w in words}

## glove를 이용해서 단어 단위를 vector로

In [10]:
test = X_train_txt_tocken[:4]
print(test)

[['-LRB-', 'The', 'film', '-RRB-', 'tackles', 'the', 'topic', 'of', 'relationships', 'in', 'such', 'a', 'straightforward', ',', 'emotionally', 'honest', 'manner', 'that', 'by', 'the', 'end', ',', 'it', "'s", 'impossible', 'to', 'ascertain', 'whether', 'the', 'film', 'is', ',', 'at', 'its', 'core', ',', 'deeply', 'pessimistic', 'or', 'quietly', 'hopeful', '.'], ['Lavishly', ',', 'exhilaratingly', 'tasteless', '.'], ['It', 'is', 'also', 'beautifully', 'acted', '.'], ['But', ',', 'like', 'Silence', ',', 'it', "'s", 'a', 'movie', 'that', 'gets', 'under', 'your', 'skin', '.']]


In [11]:
X_train_embedding = []
max_len = 40
for sentence in X_train_txt_tocken:
    temp = []
    for i,word in enumerate(sentence):
        if i == 40:
            break
        try:
            temp.append(glove[word])
        except:
            temp.append(np.random.rand(100))
    if i < 40:
        for j in range(max_len-1-i):
            temp.append(np.zeros(100))
    
    X_train_embedding.append(temp)

## y_train one hot encoding for cross entropy

In [12]:
# onehotencoding 하면 안됨
# num = np.unique(y_train_num, axis=0)
# num = num.shape[0]
# y_train = np.eye(num)[y_train_num]

## np로 바꾸기

In [13]:
X_train_embedding = np.array(X_train_embedding)

## torch로 바꾸기

In [14]:
X_train_embedding.dtype

dtype('float64')

In [17]:
X_train_embedding = torch.FloatTensor(X_train_embedding)
y_train           = torch.LongTensor(y_train_num)

In [18]:
# torch.tensor(X_train_embedding, device=cuda)

# BiLSTM_Attention으로 학습하기

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", device)

cpu와 cuda 중 다음 기기로 학습함: cuda


In [20]:
class BiLSTM_Attention(nn.Module):
    def __init__(self):
        super(BiLSTM_Attention, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, n_hidden, bidirectional=True)
        self.out = nn.Linear(n_hidden * 2, num_classes)

    # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
    def attention_net(self, lstm_output, final_state):

        hidden = final_state.view(-1, n_hidden * 2, 1)   # hidden : [batch_size, n_hidden * num_directions(=2), 1(=n_layer)]
#         print(hidden.shape)
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2) # attn_weights : [batch_size, n_step]
#         print(attn_weights.shape)
        soft_attn_weights = F.softmax(attn_weights, 1)
        # [batch_size, n_hidden * num_directions(=2), n_step] * [batch_size, n_step, 1] = [batch_size, n_hidden * num_directions(=2), 1]
        context = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)
        return context, soft_attn_weights.data.numpy() # context : [batch_size, n_hidden * num_directions(=2)]

    def forward(self, X):
        input_ = X # input : [batch_size, len_seq, embedding_dim]
#         input = self.embedding(X) # input : [batch_size, len_seq, embedding_dim]
        input_ = input_.permute(1,0,2) # input : [len_seq, batch_size, embedding_dim]
#         print
       
#        hidden_state = torch.zeros(1*2, 11544, n_hidden) # [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
#         cell_state = torch.zeros(1*2, 11544, n_hidden)# [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
#         hidden_state = torch.tensor(hidden_state,device=DEVICE)
#         cell_state = torch.tensor(cell_state,device=DEVICE)
        
        # final_hidden_state, final_cell_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]
        output, (final_hidden_state, final_cell_state) = self.lstm(input_)
        output = output.permute(1, 0, 2) # output : [batch_size, len_seq, n_hidden]
        
        attn_output, attention = self.attention_net(output, final_hidden_state)
         
        return F.softmax(self.out(attn_output),1), attention # model : [batch_size, num_classes], attention : [batch_size, n_step]

In [22]:
embedding_dim = 100 # embedding size
n_hidden = 32  # number of hidden units in one cell
num_classes = 5  # 0 or 1
batch_size  = 64
# len(X_train_embedding)

In [23]:
model = BiLSTM_Attention()
# model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
print(model)

BiLSTM_Attention(
  (lstm): LSTM(100, 32, bidirectional=True)
  (out): Linear(in_features=64, out_features=5, bias=True)
)


In [25]:
# Training
for epoch in range(500):
    optimizer.zero_grad()
    output, attention = model(X_train_embedding)
    loss = criterion(output, y_train)
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

# # Test
# test_text = 'sorry hate you'
# tests = [np.asarray([word_dict[n] for n in test_text.split()])]
# test_batch = torch.LongTensor(tests)

# # Predict
# predict, _ = model(test_batch)
# predict = predict.data.max(1, keepdim=True)[1]
# if predict[0][0] == 0:
#     print(test_text,"is Bad Mean...")
# else:
#     print(test_text,"is Good Mean!!")

# fig = plt.figure(figsize=(6, 3)) # [batch_size, n_step]
# ax = fig.add_subplot(1, 1, 1)
# ax.matshow(attention, cmap='viridis')
# ax.set_xticklabels(['']+['first_word', 'second_word', 'third_word'], fontdict={'fontsize': 14}, rotation=90)
# ax.set_yticklabels(['']+['batch_1', 'batch_2', 'batch_3', 'batch_4', 'batch_5', 'batch_6'], fontdict={'fontsize': 14})
# plt.show()

Epoch: 0010 cost = 1.600979
Epoch: 0020 cost = 1.584581
Epoch: 0030 cost = 1.573937
Epoch: 0040 cost = 1.568884
Epoch: 0050 cost = 1.561797
Epoch: 0060 cost = 1.551736
Epoch: 0070 cost = 1.540240
Epoch: 0080 cost = 1.528559
Epoch: 0090 cost = 1.517532
Epoch: 0100 cost = 1.508046
Epoch: 0110 cost = 1.499608
Epoch: 0120 cost = 1.491462
Epoch: 0130 cost = 1.483909
Epoch: 0140 cost = 1.477399
Epoch: 0150 cost = 1.471374
Epoch: 0160 cost = 1.465188
Epoch: 0170 cost = 1.458635
Epoch: 0180 cost = 1.452778
Epoch: 0190 cost = 1.447081
Epoch: 0200 cost = 1.440860
Epoch: 0210 cost = 1.435741
Epoch: 0220 cost = 1.431137
Epoch: 0230 cost = 1.427685
Epoch: 0240 cost = 1.421608
Epoch: 0250 cost = 1.416535
Epoch: 0260 cost = 1.412974
Epoch: 0270 cost = 1.408746
Epoch: 0280 cost = 1.407035
Epoch: 0290 cost = 1.401216
Epoch: 0300 cost = 1.397144
Epoch: 0310 cost = 1.394267
Epoch: 0320 cost = 1.393684
Epoch: 0330 cost = 1.386945
Epoch: 0340 cost = 1.383833
Epoch: 0350 cost = 1.387503
Epoch: 0360 cost = 1

In [26]:
torch.save(model.state_dict(),'./Bi_attention_soft_batch.pth')

In [27]:
predict, _ = model(X_train_embedding)

In [28]:
ans = 0
for i in range(len(X_train_embedding)):
    if torch.argmax(predict[i]) == y_train[i]:
        ans = ans+1
print(ans/len(X_train_embedding))

0.5753638253638254


In [39]:
torch.argmax(predict[0])

tensor(3)

In [29]:
data_test = pd.read_csv("./eval_final_open.csv")
X_test_txt = data_test["Sentence"]
X_test_txt = pd.Series.tolist(X_test_txt)

In [30]:
X_test_txt_tocken = []
for i,data in enumerate(X_test_txt):
    X_test_txt_tocken.append(word_tokenize(data))

In [31]:
X_test_embedding = []
max_len = 40
for sentence in X_test_txt_tocken:
    temp = []
    for i,word in enumerate(sentence):
        if i == 40:
            break
        try:
            temp.append(glove[word])
        except:
            temp.append(np.random.rand(100))
    if i < 40:
        for j in range(max_len-1-i):
            temp.append(np.zeros(100))
    
    X_test_embedding.append(temp)

In [32]:
X_test_embedding = np.array(X_test_embedding)
X_test_embedding = torch.FloatTensor(X_test_embedding)
X_test_embedding.shape

torch.Size([4311, 40, 100])

In [33]:
X_test_embedding[0].shape

torch.Size([40, 100])

In [34]:
test_predict,_= model(X_test_embedding)

In [45]:
for i in test_predict:
    print(torch.argmax(i).item())

2
1
2
3
1
3
3
3
3
3
2
2
1
1
3
1
3
1
3
1
1
3
1
1
2
3
2
3
1
1
3
3
1
3
1
3
1
3
2
3
3
1
3
3
1
1
3
1
2
2
3
3
1
1
3
3
3
2
2
3
3
3
3
3
1
3
1
3
3
3
3
3
3
2
2
3
3
3
3
3
1
2
2
3
3
3
1
3
2
1
1
3
3
2
3
3
1
3
2
3
1
3
2
3
3
1
1
3
1
1
2
3
3
1
1
3
1
3
1
2
1
2
1
2
3
3
3
1
3
1
1
1
1
2
3
2
1
2
3
2
1
1
1
1
1
3
2
3
2
3
3
2
2
3
3
1
1
1
3
3
3
1
2
1
3
2
1
3
2
3
3
3
2
1
3
3
1
2
3
3
3
1
3
1
3
3
2
1
3
2
3
1
3
1
1
1
1
3
1
2
1
3
1
2
1
3
3
3
3
2
1
1
3
3
2
3
3
3
1
1
3
3
3
1
3
1
1
2
1
3
1
2
1
3
2
1
1
2
2
3
3
3
3
1
3
1
3
3
1
2
1
1
1
3
2
3
2
3
3
3
1
3
1
2
1
3
3
3
3
3
1
1
3
1
1
3
3
3
3
2
1
1
3
3
3
1
1
2
2
3
1
3
3
3
3
2
1
1
2
3
3
2
1
1
3
2
3
1
3
1
3
3
1
1
1
1
1
2
2
3
3
3
3
1
2
3
3
1
2
3
1
3
1
3
3
3
3
1
1
2
2
1
1
1
3
1
1
1
1
2
3
1
3
3
1
1
2
1
1
3
3
3
3
3
2
1
2
2
3
3
1
1
2
3
1
1
3
2
1
3
3
3
3
2
3
3
1
3
1
3
2
3
3
1
1
1
1
3
1
1
3
1
1
3
3
2
1
1
1
2
1
1
2
3
3
1
1
2
3
2
1
3
1
3
3
3
3
3
3
3
1
1
1
1
1
1
1
2
2
1
3
3
3
3
2
3
2
3
1
1
1
1
3
2
1
3
3
1
3
2
1
1
1
1
2
3
1
3
2
1
3
1
1
1
1
3
2
3
3
2
1
3
1
3
1
3
1
1
3
2
3
1
1
3
2
2
1
3
3
1
